In [9]:
# 사용자의 편의성 개선
import numpy as np

In [10]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray): # 데이터가 ndarray 인스턴스만 취급되도록 한다.
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None: # grad가 None이면 자동으로 미분값을 생성한다. 모든 요소를 1로 채워서 돌려준다.
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


In [11]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [12]:
data = np.array(0.5)
a = np.ones_like(data)
print(a)

1.0


In [25]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y)) # 0차원이면 즉 그냥 정수 등이면.. 타입이 float64로 바뀐다. 따라서 as_array 함수를 이용해서 ndarray로 바꿔줘야 함

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [21]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [22]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [23]:
def square(x): # 간소화 코드
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [24]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용 가능하다.
y.backward()
print(x.grad)


x = Variable(np.array(1.0))  # OK
x = Variable(None)  # OK
x = Variable(1.0)  # NG

3.297442541400256


TypeError: <class 'float'> is not supported